In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
tensor_1 = torch.tensor([1,2,3])
tensor_2 = torch.tensor([4,5,6])
print(tensor_1 + tensor_2)

tensor([5, 7, 9])


In [5]:
tensor1 = tensor_1.to('cuda')
tensor2 = tensor_2.to('cuda')
print(tensor1 + tensor2) # all tensors must be on the same device


tensor([5, 7, 9], device='cuda:0')


In [7]:
# single gpu training

In [8]:
# model
class NeuralNetwork(torch.nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs,30),
            torch.nn.ReLU(),

            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            torch.nn.Linear(20,num_outputs),

        )


    def forward(self,x):
        logits = self.layers(x)
        return logits
# dataset
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features = X
        self.labels = y

    def __getitem__(self,index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x,one_y

    def __len__(self):
        return self.labels.shape[0]


# data loader
from torch.utils.data import DataLoader




In [9]:
#data prep
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [12]:
# gpu training on single device
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs = 2)

device = torch.device('cuda')
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        logits = model(features) # model and features on the same device
        loss = torch.nn.functional.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
    model.eval()



Epoch: 001/003 | Batch 000/003 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/003 | Train/Val Loss: 0.65
Epoch: 001/003 | Batch 002/003 | Train/Val Loss: 0.42
Epoch: 002/003 | Batch 000/003 | Train/Val Loss: 0.05
Epoch: 002/003 | Batch 001/003 | Train/Val Loss: 0.13
Epoch: 002/003 | Batch 002/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 000/003 | Train/Val Loss: 0.01
Epoch: 003/003 | Batch 001/003 | Train/Val Loss: 0.00
Epoch: 003/003 | Batch 002/003 | Train/Val Loss: 0.02


In [ ]:
# multiple gpu only in python